In [12]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Flatten, Reshape
high = 1000000
digits = len(str(high))
pad = 10

from num2words import num2words
import random

def index_list(pos):
    index_list = [0] * (pos)
    index_list.append(1)
    index_list += [0] * (10-pos-1)
    return index_list

def create_data(low,high, num):
    x_data = []
    y_data = []
    for i in range(num):
        a = random.randrange(low, high)
        b = a
        words = num2words(b)
        c = str(b).zfill(digits)  
        x_data.append(words.replace("-", " ").replace(",", "").replace(" and "," "))
        num_list = []
        for i in range(digits):
            num_list.append(index_list(int(c[i])))
        y_data.append(num_list)
    return x_data, np.array(y_data)

def append_data(x, y, x_data, y_data):
    x_data.append(x.replace("-", " ").replace(",", "").replace(" and "," "))
    c = str(y).zfill(digits) 
    num_list = [[]]
    for i in range(digits):
        num_list[0].append(index_list(int(c[i])))
    num_list = np.array(num_list)
    y_data = np.concatenate((y_data, num_list), axis=0)
    return x_data, y_data

In [13]:
x_train, y_train = create_data(0, 1000000, 60000)
x_test, y_test = create_data(0, 1000000, 40000)

for i in range(10):
    for j in range(1000, 1500, 100):
        x_train, y_train = append_data(num2words(j//100) + " hundred", j, x_train, y_train)

In [14]:
num_words = 0
for i in x_train:
    num_words += len(i.split(" "))
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_train = np.array([[0]*(pad-len(i)) + i for i in x_train])

x_test = tokenizer.texts_to_sequences(x_test)
x_test = np.array([[0]*(pad-len(i)) + i for i in x_test])

print(tokenizer.sequences_to_texts(x_train)[342])
print(x_train[342])
print(y_train[342])

vocab_size = len(tokenizer.word_index) + 1

three hundred two thousand one hundred thirty eight
[ 0  0  6  1  8  2  3  1 14  9]
[[1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]]


In [15]:
model = tf.keras.models.Sequential()
model.add(Embedding(vocab_size, pad, input_length=pad))
model.add(Flatten())
model.add(Dense(digits*10, activation=tf.nn.softmax))
model.add(Reshape((digits,10)))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.fit(x_train, y_train, epochs=35)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 10)            300       
_________________________________________________________________
flatten_2 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 70)                7070      
_________________________________________________________________
reshape_2 (Reshape)          (None, 7, 10)             0         
Total params: 7,370
Trainable params: 7,370
Non-trainable params: 0
_________________________________________________________________
Epoch 1/35
60050/60050 [==============================] - 2s 30us/sample - loss: 0.6573 - acc: 0.8552
Epoch 2/35
60050/60050 [==============================] - 2s 28us/sample - loss: 0.1040 - acc: 0.9828
Epoch 3/35
60050/60050 [==============================] - 2s 28us/sa

In [16]:
val_loss, val_acc = model.evaluate(x_test, y_test)

40000/40000 [==============================] - 1s 15us/sample - loss: 0.0100 - acc: 0.9986


In [21]:
model.save('count2.model')
new_model = tf.keras.models.load_model('count2.model')

In [23]:
x = tokenizer.texts_to_sequences(["twelve"
                                 ,"thirteen"
                                 ,"one hundred twenty three"
                                 ,"four hundred seventy two thousand two hundred twenty two"
                                 ,"two hundred thirty seven thousand one hundred forty")
                                 ,"spongebob"])
x = np.array([[0]*(pad-len(i)) + i for i in x])
predictions = new_model.predict(np.array(x))

for prediction in predictions:
    num = ""
    for i in prediction:
        num += str(i.argmax())
    print(int(num))

SyntaxError: invalid syntax (<ipython-input-23-75521ba277f0>, line 5)